### Use Python API to interact with your Canvas courses: A quick guide

This jupyter notebook is a quick intro on how you could use the Canvas API Python wrapper (https://github.com/ucfopen/canvasapi) to interact with the MIT Canvas courses you authored.

In [34]:
#import modules
from canvasapi import Canvas
import json
from os import path
import numpy as np
import pandas as pd


In [75]:
#1. MIT Canvas API endpoint:
API_URL = "https://canvas.mit.edu/api/v1"

#2. Canvas credentials
cred_file_path = "." #Specify if your credential file is elsewhere
cred_file2 = path.join(cred_file_path, "cred.json")
with open(cred_file2,'r') as cred2:
    cred_json = json.load(cred2)

API_KEY = cred_json['ACCES_TOKEN']

#3. Data upload/download paths:
data_upload = "data_upload"
data_download = "data_download"

In [36]:
# Initialize a new Canvas object
canvas = Canvas(API_URL, API_KEY)

### Get the course objects for your Canvas courses

In [37]:
#This fetches information for all courses you have access to:
courses = canvas.get_courses()
for c_ in courses[:10]:
    print(c_.id, c_.name)

3145 Canvas Core Team Sandbox
3156 MIT Canvas Resources for Instructors
4970 ODL Residential Sandbox
4974 Temporary Sandbox


### Let's  get some more information about these courses:

Let's find the number of files and assignments in each courses, and find the most recent time of update for any file or assignment.

We will print out the results as a Pandas dataframe.

In [72]:
def get_course_info(course_id):
    '''This function gets the course information and the file/assignment update times,
    by the course_id, and returns a list of courses, numbers of files and assignemtns,
    and the time of the most recent update for each course.
    
    INPUT: course_id
    
    OUTPUT: A list of 5 elements: ['course_id','course_name','Num_files','Num_assignments',
    'Most_recent_update']. Could be all None if there is an exception.
    '''
    try:
        c1 = canvas.get_course(course_id)
        course_name = c1.name

        files_ = c1.get_files()
        assn_ = c1.get_assignments()
        
        #Get the file updated times as a list
        file_utimes = [f_.updated_at.strip("Z") for f_ in files_]

        #Get the assignment updated times as a list
        assn_utimes = [a_.updated_at.strip("Z") for a_ in assn_]
        
        #Combine all timestamps, and convert the timestamps to datetime64 type
        fa_times = file_utimes + assn_utimes
        fa_times = np.array(fa_times, dtype='datetime64')
        last_update_at = fa_times.max()
        
        return [c1.id, course_name, len(file_utimes), len(assn_utimes), last_update_at]
    except Exception as e:
        print("Error {} for course {}".format(e, course_id))
        return [None, None, None, None, None]

In [73]:
course_info_list = []
course_info_cols = ['course_id','course_name','Num_files','Num_assignments','Most_recent_update']
for c_ in courses:
    course_info_list.append(get_course_info(c_.id))
    
course_info_df = pd.DataFrame.from_records(course_info_list, columns=course_info_cols)
course_info_df

,course_id,course_name,Num_files,Num_assignments,Most_recent_update
0,3145,Canvas Core Team Sandbox,11,15,2020-07-07 15:02:45
1,3156,MIT Canvas Resources for Instructors,60,0,2020-06-08 09:42:17
2,4970,ODL Residential Sandbox,24,3,2020-07-17 17:54:21
3,4974,Temporary Sandbox,23,4,2020-07-29 14:58:24


### Let's upload a file.

We will upload the test file from the data_upload folder to the "Temporary Sandbox" course (4974)
It goes in the "unfiled" folder under the "files" section of the course

In [33]:
file_to_upload = "test_file" #name of the file in the upload folder
file_ = path.join(data_upload, file_to_upload) #define the file path

course_id = 4974 #be sure of the course id
c1 = canvas.get_course(course_id) #get the course object
c1.upload(file_)

(True,
 {'location': 'https://canvas.mit.edu/api/v1/files/546871?include%5B%5D=enhanced_preview_url',
  'instfs_uuid': '6be07789-2364-419c-a632-04c53312685f',
  'id': 546871,
  'uuid': '92aE9TJOPNlq7II9lPyTNiIZiFtXFH8N8Z7i2Hgo',
  'folder_id': 276045,
  'display_name': 'test_file',
  'filename': 'test_file.txt',
  'upload_status': 'success',
  'content-type': 'text/plain',
  'url': 'https://canvas.mit.edu/files/546871/download?download_frd=1&verifier=92aE9TJOPNlq7II9lPyTNiIZiFtXFH8N8Z7i2Hgo',
  'size': 55,
  'created_at': '2020-07-29T14:57:39Z',
  'updated_at': '2020-07-29T14:57:39Z',
  'unlock_at': None,
  'locked': False,
  'hidden': False,
  'lock_at': None,
  'hidden_for_user': False,
  'thumbnail_url': None,
  'modified_at': '2020-07-29T14:57:39Z',
  'mime_class': 'text',
  'media_entry_id': None,
  'locked_for_user': False,
  'preview_url': '/courses/4974/files/546871/file_preview?annotate=0'})

### Conclusions

This concludes the quick demo. Follow the URLs below (especially the course section) to see example use cases. Let us know if you are using it, and if you have any questions.

#### References:
1. https://canvasapi.readthedocs.io/ for the entire Canvas API Python wrapper
2. https://canvasapi.readthedocs.io/en/stable/course-ref.html for the section on courses
3. https://canvas.instructure.com/doc/api/index.html for the Canvas API documentation and keywords that may be missing from the Python wrapper documentation